In [1]:
import pandas as pd
import pymysql
import re
from datetime import datetime
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from py.ankety_correction import *

# Импорт данных

In [ ]:
# Импорт физических лиц из базы mysql
def import_table(db_host, db_user, db_password, db_name, table_name):
    # Подключение к базе данных
    connection = pymysql.connect(host=db_host,
                                 user=db_user,
                                 password=db_password,
                                 database=db_name,
                                 cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            # SQL запрос для выбора всех данных из таблицы
            sql_query = f"SELECT * FROM {table_name}"
            cursor.execute(sql_query)

            # Получение результатов запроса
            rows = cursor.fetchall()

            # Преобразование результатов в DataFrame
            df = pd.DataFrame(rows)
            return df
    finally:
        connection.close()

df_fiz = import_table('localhost', 'user', 'password', 'alumny_db', 'persons')

In [55]:
file_name = 'lids_22_02_24_по_11_03_24'

In [56]:
#Выгрузка из Битрикса
url_lids = r'C:\Users\0478\Documents\Repositories\base\lids\{}.csv'.format(file_name)
#Выгрузка из 1С: необходимо выгружать каждый раз более ранние записи
url_1c = r"C:\Users\0478\Documents\Repositories\base\1C\1C_2023_po_22_02_24.xlsx"
#Выгрузка из базы анкет выпускника
url_ank = r"C:\Users\0478\Documents\Repositories\base\excel_data\\02_lids_22_02_24.xlsx"

df_lid = pd.read_csv(url_lids, sep=';', dtype='str')
df_lid = df_lid[df_lid['Название лида'] == 'Заполнение CRM-формы "Анкета выпускника"'].reset_index(drop=True)
df_1c = pd.read_excel(url_1c, header=0, dtype='str')
df_ank = pd.read_excel(url_ank, header=0, dtype='str')


# Предобработка данных

In [57]:
#Приводим все таблицы к нижнему регистру и удаляем в строковых колонках пробелы и запятые по краям   
df_lid = lower_strip(df_lid, df_full=True)
df_fiz = lower_strip(df_fiz, df_full=True)
df_1c = lower_strip(df_1c, df_full=True)
df_ank = lower_strip(df_ank, df_full=True)

In [59]:
# Коррекция анкет

phone_correction(df_lid, 'Рабочий телефон') # Коррекция телефонов
df_lid = drop_columns_ank(df_lid) # Удаление лишних колонок
date_correction(df_lid, 'Дата создания') # Коррекция дат
date_correction(df_lid, 'Дата рождения') # Коррекция дат
df_lid = citizenship_correction_ank(df_lid, True) # Коррекция гражданства
df_lid = residence_correction_ank(df_lid, True) # Коррекция места жительства
df_lid = language_correction(df_lid, True) # Коррекция языков
forma_ob_correction(df_lid) # Коррекция формы обучения


# Добавление уникальных id

In [66]:
# проставляем id, в список save_list_id_1 сохраняются проставленные id для того, чтобы позже произвести проверку 
df_ank_id, save_list_id_1 = id_find(df_lid_temp, df_fiz, 'Рабочий e-mail', 'Email')
print(f'Заполнено {df_ank_id[df_ank_id['id'] != ''].shape} строк')
print(f'Осталось не заполненными {df_ank_id[df_ank_id['id'] == ''].shape} строк')

Заполнено (15, 51) строк
Осталось не заполненными (164, 51) строк


In [72]:
# Проверяем максимальный индекс, чтобы отслеживать возможные ошибки
fiz_index_max_before_update = df_fiz.index.max()
print(f'Максимальный индекс в df_fiz перед добавлением новых строк: {fiz_index_max_before_update}')
print(f'Размерность df_fiz перед добавлением новых строк: {df_fiz.shape}')

# Добавляем в физлица тех, кого там нет
df_fiz = add_new_row_to_fiz_from_ank(df_ank_id, df_fiz)
print(f'Размерность df_fiz после добавления новых строк: {df_fiz.shape}')
print(f'Максимальный индекс в df_fiz после добавления новых строк: {df_fiz.index.max()}')

Максимальный индекс в df_fiz перед добавлением новых строк: 39102
Размерность df_fiz перед добавлением новых строк: (39103, 25)
Размерность df_fiz после добавления новых строк: (39267, 25)
Максимальный индекс в df_fiz после добавления новых строк: 39266


In [74]:
# Снова проставляем id в анкеты
df_ank_id, save_list_id_2 = id_find(df_ank_id, df_fiz, 'Рабочий e-mail', 'Email')
print(f'Заполнено {df_ank_id[df_ank_id['id'] != ''].shape} строк')
print(f'Осталось не заполненными {df_ank_id[df_ank_id['id'] == ''].shape} строк')

Заполнено (179, 51) строк
Осталось не заполненными (0, 51) строк


In [76]:
# Добавление информации из анкет в физлица
df_fiz2 = ank_to_fiz(df_fiz, df_ank_id, 'phone', 'Рабочий телефон').copy()
df_fiz2 = ank_to_fiz(df_fiz2, df_ank_id, 'languages', 'Какими иностранными языками Вы владеете?')
df_fiz2 = ank_to_fiz(df_fiz2, df_ank_id, 'company', 'Название компании')
df_fiz2 = ank_to_fiz(df_fiz2, df_ank_id, 'position', 'Должность')
df_fiz2 = ank_to_fiz(df_fiz2, df_ank_id, 'citizenship', 'Ваше гражданство')
df_fiz2 = ank_to_fiz(df_fiz2, df_ank_id, 'residential_address', 'Место жительства')
df_fiz2 = ank_to_fiz(df_fiz2, df_ank_id, 'want_to_assotiation', 'Хотите ли вступить в Ассоциацию выпускников РГГУ?')
df_fiz2 = ank_to_fiz(df_fiz2, df_ank_id, 'disability_category', 'Есть ли у Вас инвалидность')
df_fiz2 = ank_to_fiz(df_fiz2, df_ank_id, 'faculty', 'Факультет и направление')

df_fiz2 = unsubscribe(df_fiz2, df_ank_id, 'unsubscribe', 'Хотели бы вы получать рассылку с вакансиями, стажировками и анонсами карьерных мероприятий?', 'no_vacancy')
df_fiz2 = unsubscribe(df_fiz2, df_ank_id, 'unsubscribe', 'Хотели бы вы получать информацию об открытии наборов на обучение по программам профессиональной переподготовки и повышения квалификации', 'no_dpo')